In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
os.chdir("/sfs/qumulo/qhome/jtb3sud/PARCv2")
import PARC.data.data_meta_learning as data
#from parc import misc, metrics, visualization
from PARC.model import model_meta_learning as model


2024-05-09 13:17:16.480467: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-09 13:17:21.942103: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1636] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78791 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:b7:00.0, compute capability: 8.0


# Data pipeline

In [2]:
# Get data and normalization
state_seq_whole, vel_seq_whole = data.clip_raw_data(sequence_length = 52, n_state_var = 3, purpose = "diff_training", folder_path = '/scratch/jtb3sud/meta_data/tnt/training', image_size = (128, 208))
state_seq_norm = data.data_normalization(state_seq_whole,3)
vel_seq_norm = data.data_normalization(vel_seq_whole,2)

Processing 183118.npy
Processing 183135.npy
Processing 183127.npy
Processing 183134.npy
Processing 183115.npy
Processing 183132.npy
Processing 183114.npy
Processing 183123.npy
Processing 183140.npy
Processing 183130.npy
Processing 183133.npy
Processing 183129.npy
Processing 183111.npy
Processing 183105.npy
Processing 183102.npy
Processing 183113.npy
Processing 183137.npy
Processing 183136.npy
Processing 183119.npy
Processing 183144.npy
Processing 183110.npy
Processing 183112.npy
Processing 183122.npy
Processing 183104.npy
Processing 183128.npy
Processing 183141.npy
Processing 183145.npy
Processing 183103.npy
Processing 183107.npy
Processing 183106.npy
Processing 183121.npy
Processing 183108.npy
Processing 183142.npy
Processing 183109.npy
Processing 183125.npy


# Training


### Stage 1: Differentiator training

In [3]:
# Create tf.dataset
dataset_input = tf.data.Dataset.from_tensor_slices((state_seq_norm[0][:,:,:,:3],vel_seq_norm[0][:,:,:,:2]))
dataset_label = tf.data.Dataset.from_tensor_slices((state_seq_norm[0][:,:,:,-3:],vel_seq_norm[0][:,:,:,-2:]))
dataset = tf.data.Dataset.zip((dataset_input, dataset_label))
dataset = dataset.shuffle(buffer_size = 2192) 
dataset = dataset.batch(8)

In [4]:
print(state_seq_whole.shape, vel_seq_whole.shape)

(339, 128, 208, 156) (339, 128, 208, 104)


In [5]:
print(state_seq_norm[0].shape, vel_seq_norm[0].shape)

(339, 128, 208, 156) (339, 128, 208, 104)


In [6]:
dataset

<_BatchDataset element_spec=((TensorSpec(shape=(None, 128, 208, 3), dtype=tf.float64, name=None), TensorSpec(shape=(None, 128, 208, 2), dtype=tf.float64, name=None)), (TensorSpec(shape=(None, 128, 208, 3), dtype=tf.float64, name=None), TensorSpec(shape=(None, 128, 208, 2), dtype=tf.float64, name=None)))>

In [7]:
tf.keras.backend.clear_session()
# step size needs to be changed to 1/total number of steps
#parc = model.PARCv2(n_state_var = 3, n_time_step = 1, step_size= 1/60, solver = "rk4", mode = "differentiator_training")
parc = model.PARCv2(n_state_var = 3, n_time_step = 1, step_size= 1/52, solver = "rk4", mode = "differentiator_training", image_size = (128, 208))
parc.differentiator.load_weights('/sfs/qumulo/qhome/jtb3sud/PARCv2/Pretrained_Weights/em/parc2_diff_rk4.h5')
parc.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00001, beta_1 = 0.9, beta_2 = 0.999))
parc.fit(dataset, epochs = 1, shuffle = True)

2024-05-09 13:19:32.258026: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8904
2024-05-09 13:19:40.685727: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f78d04126c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-09 13:19:40.685789: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-SXM4-80GB, Compute Capability 8.0
2024-05-09 13:19:40.798842: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-09 13:19:41.656383: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


43/43 [==============================] - 97s 979ms/step - total_loss: 18035.5371


In [8]:
parc.differentiator.save_weights('/sfs/qumulo/qhome/jtb3sud/PARCv2/Pretrained_Weights/tnt/parc2_diff_rk4.h5')

### Stage 2: Data-driven integration training

In [10]:
# Pretrain integrator
tf.keras.backend.clear_session()
parc = model.PARCv2(n_state_var = 3, n_time_step = 1, step_size= 1/52, solver = "rk4", mode = "integrator_training", image_size = (128, 208))
parc.integrator.load_weights('/sfs/qumulo/qhome/jtb3sud/PARCv2/Pretrained_Weights/em/parc2_int_rk4.h5')
parc.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00001, beta_1 = 0.9, beta_2 = 0.999))
parc.fit(dataset, epochs = 1, shuffle = True)

43/43 [==============================] - 57s 496ms/step - total_loss: 17915.3359


In [11]:
parc.integrator.save_weights('/sfs/qumulo/qhome/jtb3sud/PARCv2/Pretrained_Weights/tnt/parc2_int_rk4.h5')